# Import the required dependencies

In [ ]:
import pandas as pd
import numpy as np

  Cloning https://github.com/tensorflow/docs to /tmp/pip-req-build-s44dn0w2
  Running command git clone -q https://github.com/tensorflow/docs /tmp/pip-req-build-s44dn0w2
  Created wheel for tensorflow-docs: filename=tensorflow_docs-0.0.05dfd89e64eba8083f1978bdb826b7ca7c42a8bca_-cp36-none-any.whl size=146356 sha256=a8c09db72cdf0a7980d22d340d06c0b42308f7494c1d5245fdbb700f94c01db8
  Stored in directory: /tmp/pip-ephem-wheel-cache-jo3g1oj4/wheels/eb/1b/35/fce87697be00d2fc63e0b4b395b0d9c7e391a10e98d9a0d97f
Successfully built tensorflow-docs


# 1. Loading the data

In [ ]:
# Read the dataset in excel format
# index_col is the column that should be used for indexing the students (student id)
dataset = pd.read_excel('dataset.xlsx', index_col='aluno')

## Understanding the dataset structure
First it's important to take a look on how the data is structured

In [ ]:
dataset

,idade,nomeconcelho,paigse,painesc,paiprofissao,maegse,maenesc,maeprofissao,sexo,Desistencia,ALGEBRA LINEAR,ANALISE MATEMATICA I,ANALISE MATEMATICA II,ARQUITETURA DE COMPUTADORES,INGLES E TECNICAS DE COMUNICAC?O I,INGLES E TECNICAS DE COMUNICAC?O II,INTRODUC?O A ENGENHARIA INFORMATICA,LABORATORIO INTEGRADO I,LOGICA COMPUTACIONAL,METODOLOGIAS DE PROGRAMAC?O I,SEMINARIO DE INFORMATICA,SISTEMAS DIGITAIS
aluno,,,,,,,,,,,,,,,,,,,,,,
26033751975,27,VILA REAL,Desconhecido / N?o tem,4? ano de escolaridade (antiga 4? classe),Desconhece,Desconhecido / N?o tem,4? ano de escolaridade (antiga 4? classe),Desconhece,NaN,0,16,12,0,14,14,11.0,16.0,13,11.0,10,0,14
25621666917,29,SEM CONCELHO,Operarios de instalac?es e maquinas,4? ano de escolaridade (antiga 4? classe),Desconhece,Especialistas das profiss?es intelectuais e ci...,"Ensino superior (bacharelato, licenciatura)",Desconhece,M,1,12,0,0,10,0,0.0,0.0,0,0.0,0,0,0
25697772716,27,SEM CONCELHO,Desconhecido / N?o tem,Ensino secundario complementar ou equivalente ...,Outra situac?o,Desconhecido / N?o tem,Ensino secundario complementar ou equivalente ...,Outra situac?o,NaN,0,16,11,0,14,0,0.0,14.0,13,0.0,14,0,12
25796153383,29,VILA REAL,Pessoal das Forcas Armadas,9? ano de escolaridade (antigo 5? ano),Desconhece,Especialistas das profiss?es intelectuais e ci...,Ensino secundario complementar ou equivalente ...,Desconhece,M,0,13,10,0,17,14,14.0,12.0,10,10.0,13,0,13
25896390289,31,SEM CONCELHO,Outra situac?o,4? ano de escolaridade (antiga 4? classe),Desconhece,"Aluno, estudante",9? ano de escolaridade (antigo 5? ano),Desconhece,NaN,0,13,11,0,10,14,13.0,13.0,16,14.0,15,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93131223108,22,SANTA MARTA DE PENAGUI?O,Trabalhador por conta de outrem,Ensino secundario complementar ou equivalente ...,Desconhece,Trabalhador por conta de outrem,Ensino secundario complementar ou equivalente ...,Desconhece,M,0,0,0,0,0,0,0.0,0.0,0,0.0,0,0,18
92826799912,22,SEM CONCELHO,Trabalhador por conta de outrem,9? ano de escolaridade (antigo 5? ano),"Trabalhadores qualificados da industria, const...",Desempregado,9? ano de escolaridade (antigo 5? ano),Trabalhadores n?o qualificados,M,0,0,0,0,0,0,0.0,0.0,0,0.0,0,0,17
92928893057,21,SEM CONCELHO,Trabalhador por conta propria como isolado,9? ano de escolaridade (antigo 5? ano),Pessoal administrativo,Trabalhador por conta propria como isolado,Ensino secundario complementar ou equivalente ...,Pessoal administrativo,NaN,0,0,0,0,0,0,0.0,0.0,0,0.0,0,0,10


# 2. Cleaning the dataset

## 2.1 Handling missing values
First we need to find out which columns have missing values (NaN)

In [ ]:
# Here we find out that the sex column is the only one with missing values
missing_values_columns=dataset.columns[dataset.isna().any()].tolist()
print(f"Columns with missing values (NaN): {missing_values_columns}")

Columns with missing values (NaN): ['sexo']


### 2.1.1 Filling the 'Sex' Column

Let's find how much data is missing

In [ ]:
missing = dataset["sexo"].isnull().sum(axis=0)
total = dataset.shape[0]

print(f"Percentage of missin values in the column 'sexo': {(missing*100)/total:.2f} %")

Percentage of missin values in the column 'sexo': 43.38 %


For filling the missing values of this column, we will use a simple yet effective strategy. We will replace all the missing values with the most frequent one. For this we can use [SimpleImputer](https://scikit-learn.org/stable/modules/generated/sklearn.impute.SimpleImputer.html) from scikit-learn.

In [ ]:
# Import the 'SimpleImputer' module
from sklearn.impute import SimpleImputer


# Create an imputer object with the 'most_frequent' strategy.
# Here we also specify that the missing values are 'NaN'
# verbose 0 means no logging
imp_most_frequent = SimpleImputer(missing_values=np.nan, strategy='most_frequent', verbose=0)

# We apply this method to the sex column
# We need to perform the reshape because this method expects a 2D array
# and we are only passing 1 column. So we need to make it a 2D array with only 1 column and n rows.
dataset["sexo"] = imp_most_frequent.fit_transform(dataset["sexo"].values.reshape(-1, 1))

## Other columns
With the above code we found out that only the sex column has 'NaN' values. However, when we first performed the inspection we could notice that many columns (regarding parent's information) have a custom string for missing values. We still can apply the above strategy to this columns.

How can we solve this?

A simple strategy is to first manually analyze the unknow string value for each column.

In [ ]:
# We manually construct a dictionary where each column (key) has the unknow value string (value)
target_cols = {"nomeconcelho": "SEM CONCELHO",
               "paigse": "Desconhecido / N?o tem",
               "painesc" : "Desconhecido",
               "paiprofissao" : "Desconhece",
               "maegse" : "Desconhecido / N?o tem",
               "maenesc" : "Desconhecido", 
               "maeprofissao" : "Desconhece"}


In [ ]:
# Here we iterate over (key, value) pair in the previous dictionary
# Each key ('column') is the column to be transformed
# Each value ('unknow_value_string') is the value to replace
for column, unknow_value_string in target_cols.items():
  # Create custom imputer object for each column 
  imp_most_frequent = SimpleImputer(missing_values = unknow_value_string, strategy='most_frequent', verbose = 0)
  # Perform the replacement (reshape is required to make the data a (n,1) array)
  dataset[column] = imp_most_frequent.fit_transform(dataset[column].values.reshape(-1, 1))


## 2.2 Transforming Categorical Columns
First let's find out which columns have categorical (non-numeric) values.

In [ ]:
non_numeric_columns = dataset.columns.difference(dataset._get_numeric_data().columns).to_list()
print(f"Columns with categorical data: {non_numeric_columns}")

Columns with categorical data: ['maegse', 'maenesc', 'maeprofissao', 'nomeconcelho', 'paigse', 'painesc', 'paiprofissao', 'sexo']


Now we should encode each of this columns to numeric. For this we will use the [One-Hot encoding technique](https://en.wikipedia.org/wiki/One-hot). Pandas provides us a high level API for this, where we only need to specify the list of columns that should be encoded (we already got this list).

In [ ]:
# Perform 'OneHotEncoding' for the all the categorical columns
dataset = pd.get_dummies(dataset, columns=non_numeric_columns)

We also need to split our dataframe, called `dataset`, into `X` and `y`:
*   `X` should contain the dataset features;
*   `y` should contain the dataset labels.



In [ ]:
# Extract the 'Dropout' ('Desistencia') column from X (features) to be used as target (y)
X = dataset[dataset.columns.difference(["Desistencia"])]
y = dataset[["Desistencia"]]
# Make sure y is an array
y = np.array(y).ravel()

# Save the feature names
features_names = X.columns.values


## 2.3 Feature Scaling

For neural networks it's really important and common practise to rescale the data to a common range. For this we can use [StandardScaler](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html) from scikit-learn, which removes the `mean`  and sets the `standard deviation` to 1.


Notes:

This procedure does not affect `RandomForests` performance.

In [ ]:
# Import the scaler for scikit-learn
from sklearn.preprocessing import StandardScaler

# Scale the features
X = StandardScaler().fit_transform(X)


## 2.4 Splitting the dataset
It's mandatory to not use the same data for training and validating the model. Therefore, K-Fold Cross-validation will be used. We can use the scikit-learn API [StratifiedKFold](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.StratifiedKFold.html).

In [ ]:
# Import the k-fold from scikit-learn
from sklearn.model_selection import StratifiedKFold
# The stratified means that the number of positive labels are distributed evenly trough the folds.

# Object that performs shuffling and data splitting (10 folds)
cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=0)

# 3. Building the models

## 3.1 Random Forest
To implement `RandomForests` we can use the scikit-learn [RandomForestClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html?highlight=randomforestclassifier).

In [ ]:
# Import the classifier and cross-validation method
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_validate

# Create baseline (no parameters tuning (random_state is only for reproducible results))
model = RandomForestClassifier(random_state=0)

# Establish metrics that should be calculated
scoring=['accuracy', 'roc_auc']

# Perform cross-validation on the data
# We pass the model, the data, scoring metrics and our custom cross-validation strategy
results = cross_validate(model, X, y, scoring = scoring , cv = cv)

# Results is a dictionary and we can extract the metrics results
accuracy = results["test_accuracy"]
auc = results["test_roc_auc"]

# Print `mean` values from metrics and their `std` (standard deviation)
print(f"Accuracy: {accuracy.mean():.2f} (+/- {accuracy.std():.2f})")
print(f"AUC: {auc.mean():.2f} (+/- {auc.std():.2f})")




Accuracy: 0.85 (+/- 0.05)
AUC: 0.92 (+/- 0.06)


## Inspecting the most important features

`RandomForests` can be used to inspect the most important features in a dataset. 

Please note that this method can be misleading for high cardinality features (many unique values). Please check [PermutationImportance](https://scikit-learn.org/stable/modules/generated/sklearn.inspection.permutation_importance.html) as an alternative.

In [ ]:
# Model needs to be re-fitted (trained again)
model.fit(X,y)

# Extract the feature importance
importances = model.feature_importances_

# Extract indices for sorting the importances
indices = np.argsort(importances)[::-1]

# Number of best features to show
how_many_features = 8

print(f"Feature ranking (top {how_many_features}):")
for f in range(how_many_features):
    print(f"{f+1}. feature {features_names[indices[f]]} ({importances[indices[f]]})")


Feature ranking (top 8):
1. feature SISTEMAS DIGITAIS (0.08721715124994979)
2. feature ARQUITETURA DE COMPUTADORES (0.081328988253291)
3. feature idade (0.0775741143167571)
4. feature LABORATORIO INTEGRADO I (0.05880461380615652)
5. feature ALGEBRA LINEAR (0.05051097417356697)
6. feature ANALISE MATEMATICA I (0.04904704300055357)
7. feature METODOLOGIAS DE PROGRAMAC?O I (0.040738948010107315)
8. feature LOGICA COMPUTACIONAL (0.04072276826048913)


## Tuning the parameters

We can try to improve our model by tuning it's parameters. One way to achieve this is with [GridSearch](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html). This algorithm performs an exhaustive search over specified parameters for a model.

In [ ]:
# Import the model
from sklearn.model_selection import GridSearchCV

# Define the parameter grid
parameters = {'n_estimators': [ 50, 70, 90, 110],
              'max_depth':[7, 10, 13, 16, 20]}

# Build a new GridSearch model based on the previous RandomForest model (GridSearch itself implements the same interface as the RandomForest model)
model = GridSearchCV(model, parameters, cv=cv, scoring=scoring, refit='roc_auc')
# The refit parameter indicates the metric that should be used to choose the best model

# Fit the data on model for performing the cross-validation
model.fit(X, y)

# Show best parameters
print(f"Best parameters: {model.best_params_}")

# Acess cross-validation results
results = model.cv_results_ 

# Extract cross-validation metric results of the best model
mean_acc = results["mean_test_accuracy"][model.best_index_]
std_acc = results["std_test_accuracy"][model.best_index_]

mean_auc = results["mean_test_roc_auc"][model.best_index_]
std_auc = results["std_test_roc_auc"][model.best_index_]

# Print the results
print(f"Accuracy: {mean_acc:.2f} (+/- {std_acc:.2f})")
print(f"AUC: {mean_auc:.2f} (+/- {std_acc:.2f})")


Best parameters: {'max_depth': 16, 'n_estimators': 50}
Accuracy: 0.86 (+/- 0.03)
AUC: 0.93 (+/- 0.03)


## 3.2 Neural Networks
To implement neural networks we use [Tensorflow Keras](https://www.tensorflow.org/guide/keras). This API allows us to build complex neural networks easily.

In [ ]:
# Import tensorflow
import tensorflow as tf

# Define model architecture
model = tf.keras.Sequential([
    tf.keras.layers.Dense(8, input_shape=(X.shape[1],), activation='relu' ),
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dense(1, activation = 'sigmoid') # apenas um neurónio para output de 0 ou 1
  ])

# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(),
                loss=tf.keras.losses.BinaryCrossentropy(),
                metrics=[tf.keras.metrics.Accuracy(), 
                         tf.keras.metrics.AUC()])

# Create lists for holding the metric values
acc_per_fold=[] # accuracy
auc_per_fold=[] # AUC

# Tensorflow isn't compatible by default with our custom cross-validation object. So we need to perform the data splits and the cross-validation itself manually.
# To do this we use a for loop and call the split method of our cross-validation object, which will return indices for training and testing in each iteration.
# Then we use this indices for splitting the data.
for train_idx, test_idx in cv.split(X, y):

  # Train the model with the training indices
  model.fit(X[train_idx], y[train_idx], epochs=75, verbose=0)
    
  # Evaluate the model with the testing indices
  scores = model.evaluate(X[test_idx], y[test_idx], verbose=0)

  # Save the accuracy value on the test data
  acc_per_fold.append(scores[1])
  # Save the AUC value on the test data
  auc_per_fold.append(scores[2])

In [ ]:
# Transform the lists into numpy arrays for calculations
acc_per_fold=np.array(acc_per_fold)
auc_per_fold=np.array(auc_per_fold)

# Display the results
print(f"Accuracy: {acc_per_fold.mean():.2f} (+/- {acc_per_fold.std():.2f})")
print(f"AUC: {auc_per_fold.mean():.2f} (+/- {auc_per_fold.std():.2f})")

Binary accuracy: 0.95 (+/- 0.05)
Loss: 0.22 (+/- 0.14)
AUC: 0.99 (+/- 0.02)
Precision: 0.87 (+/- 0.15)
Recall: 0.97 (+/- 0.07)
